In [25]:
# from langchain.chains.question_answering.map_reduce_prompt import messages
# !pip install python-dotenv
# !pip install -U openai langchain langchain-openai
!pip install -U langchain-ollama
!pip install langchain_community
!pip install pypdf
!pip install faiss-cpu
!pip install chromadb
!pip install langchain-teddynote
!pip install googletrans

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached googletrans-4.0.2-py3-none-any.whl.metadata (10 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached h2-4.2.0-py3-none-any.whl.metadata (5.1 kB)
  Using cached hyperframe-6.1.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached hpack-4.1.0-py3-none-any.whl.metadata (4.6 kB)
Using cached googletrans-4.0.2-py3-none-any.whl (18 kB)
Us

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
  Using cached googletrans-4.0.0rc1-py3-none-any.whl
  Using cached httpx-0.13.3-py3-none-any.whl.metadata (25 kB)
  Using cached hstspreload-2025.1.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl.metadata (3.2 kB)
  Using cached idna-2.10-py2.py3-none-any.whl.metadata (9.1 kB)
  Using cached rfc3986-1.5.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached httpcore-0.9.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached h11-0.9.0-py2.py3-none-any.whl.metadata (8.1 kB)
  Using cached h2-3.2.0-py2.py3-none-any.whl.metadata (32 kB)
  Using cached hyperframe-5.2.0-py2.py3-none-any.whl.metadata (7.2 kB)
  Using cached hpack-3.0.0-py2.py3-none-any.whl.metadata (7.0 kB)
Using cached httpx-0.13.3-py3-none-any.whl (55 kB)
Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
Using cached httpcore-0.9.1-py3-none-any.whl (42 kB)
Using cached idna-2.10-py2.py3-none-any.whl (58

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anthropic 0.47.2 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.
chromadb 0.6.3 requires httpx>=0.27.0, but you have httpx 0.13.3 which is incompatible.
langgraph-sdk 0.1.53 requires httpx>=0.25.2, but you have httpx 0.13.3 which is incompatible.
langsmith 0.3.9 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.
ollama 0.4.7 requires httpx<0.29,>=0.27, but you have httpx 0.13.3 which is incompatible.
openai 1.63.2 requires httpx<1,>=0.23.0, but you have httpx 0.13.3 which is incompatible.
jupyterlab 4.2.5 requires httpx>=0.25.0, but you have httpx 0.13.3 which is incompatible.


In [5]:
from dotenv import load_dotenv
import os
from langchain_teddynote import logging
# import openai
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter, TextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.indexes.vectorstore import VectorstoreIndexCreator
from langchain.vectorstores import Chroma
import time
import re
from translate import Translator

In [6]:
load_dotenv()
logging.langsmith("mammothLangchain", set_enable=True)

LangSmith 추적을 시작합니다.
[프로젝트명]
mammothLangchain


In [7]:
def pdf_loader(file_path):
    loader = PyPDFLoader(file_path)
    return loader

In [8]:
def show_pdf(loader):
    documents = loader.load()

    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)
    context = [doc.page_content for doc in docs]
    print(context)

In [9]:
def translate_file_name(file_name):
    translator = Translator(to_lang="en", from_lang="ko")
    en_file_name = translator.translate(file_name)
    return en_file_name

In [10]:
def get_collection_name(file_name: str) -> str:
    # 파일 이름을 영어로 번역
    en_file_name = translate_file_name(file_name)
    # 두 개의 연속된 마침표(..)를 포함할 경우 삭제
    en_file_name = re.sub(r'/./.+', '.', en_file_name)
    # 영숫자 문자, 밑줄(_) 또는 하이픈(-)만 남기고 나머지 기호 삭제
    en_file_name = re.sub(r'[^a-zA-Z0-9_-]', '', en_file_name)
    # 시작과 끝에 기호나 숫자가 있다면 기호 삭제
    en_file_name = re.sub(r'^[^a-zA-Z0-9]+', '', en_file_name)
    en_file_name = re.sub(r'[^a-zA-Z0-9]+$', '', en_file_name)
    # 길이 3자 미만이면 글자 추가, 63자 초과하면 63자로 자르기
    if len(en_file_name) < 3:
        en_file_name = en_file_name.ljust(3, 'a')
    elif len(file_name) > 63:
        en_file_name = en_file_name[:63]

    return en_file_name

In [11]:
# def create_vector_store(file_name, loader, loaders=None, embeddings, vectorstore_path):
    # docs = []
    # for loader in loaders:
    #     docs.extend(loader.load())
    #     print(docs)
    #
    # sub_docs = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100).split_documents(docs)
    # print(sub_docs)
    #
    # vectorstore = Chroma.from_documents(
    #     sub_docs,
    #     embedding=embeddings,
    #     kwargs={
    #         "persist_directory":vectorstore_path,
    #         # "collection_name":get_collection_name(file_name),
    #     }
    # )
    #
    #
    #
    # VectorstoreIndexCreator(
    #     vectorstore_cls=Chroma,
    #     embedding=embeddings,
    #     vectorstore_kwargs={
    #         "persist_directory":vectorstore_path,
    #         # "collection_name":get_collection_name(file_name),
    #     }
    #     ).from_loaders([loader])

In [25]:
# 벡터 스토어를 새로 만들 경우, 저장하기 까지 시간이 걸려서 메모리에 저장
def create_vectorstore(file_id, file_texts, embeddings, vectorstore_path):
    document = (
        file_texts,
        file_id,
    )

    vectorstore = Chroma(
        persist_directory=vectorstore_path,
        embedding_function=embeddings,
    ).add_texts(document)

    index = VectorStoreIndexWrapper(vectorstore=vectorstore)

    return index

In [13]:
def load_vectorstore(vectorstore_path):
    index = VectorStoreIndexWrapper(vectorstore=vectorstore_path)
    return index

In [26]:
def get_vectorstore(file_name, loader, embeddings, vectorstore_path, db_already_exist):

    if db_already_exist:
        index = load_vectorstore(embeddings)

    elif not db_already_exist:
        index = create_vector_store(file_name, loader, embeddings, vectorstore_path)

    return index

In [15]:
def set_model(model):
    chat = ChatOllama(
        model=model,
        temperature=0.1,
        num_predict=512,
        top_p=1,
        mirostat_tau=0.4,
        # CPU 코어수
        num_thread=None,
    )
    return chat

In [16]:
def set_chat():
    file_path = "C:/Users/user/Documents/GitLab/Test_Dawoo/company_ask/doc/company_info/high_quelity/타임소프트 회사소개서.pdf"
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    print(file_name)
    vectorstore_path = "./vectorstore"
    model = 'llama3.2-bllossom-kor-3B:latest'
    embeddings = OllamaEmbeddings(model="mxbai-embed-large")
    #embeddings = HuggingFaceEmbeddings()

    # env_loader()
    loader = pdf_loader(file_path)
    # show_pdf(loader)
    index = get_vectorstore(
        file_name=file_name,
        loader=loader,
        embeddings = embeddings,
        vectorstore_path=vectorstore_path,
        db_already_exist=False,
    )
    chat = set_model(model=model)

    return chat, index

In [17]:
def chat_input(index, user_input, chat):
    return index.query( user_input, llm=chat, )

In [18]:
#테스트 셀

#파일 출력 테스트
file_path = "C:/Users/user/Documents/GitLab/Test_Dawoo/company_ask/doc/company_info/high_quelity/타임소프트 회사소개서.pdf"
pdf = pdf_loader(file_path)
show_pdf(pdf)


#번역 및 파일 이름 가공 테스트
# file_name = input()
# print(file_name)
# en_file_name = translate_file_name(file_name)
# print(en_file_name)
# collection_name = get_collection_name(en_file_name)
# print(collection_name)


# #스트리밍 테스트
# chat, index = set_chat()
# user_input = '안녕?'
# chat_text = chat_input(
#     chat=chat,
#     index=index,
#     user_input=user_input,
# )
# print(f"chat: {chat_text}")
# time.sleep(0.5)

['01AIoT 빅데이터 융합 플랫폼 제공과 공급02빅데이터 융합 플랫폼 기반의 생체인증 SaaS 서비스 제공03Cloud Native MSA 기반의 개발 환경과 운영 환경 제공과 공급', '고객의생각과 이상을현실로 !\n외부고객 : Customer, 내부고객 : Staff, Stockholder현재, 요구, 가려운 곳미래', '157 억원http://www.timesoft.co.kr', '대표이사경영지원실전략기술팀DevOpsCloud MSA기업부설연구소Cloud Native MSABio SecurityMammoth Data플랫폼 그룹IaaSPaaS서비스 그룹ezPass 서비스MSA 서비스Bio Security매머드 사업본부AIoTService Plan공공부분금융부분통신부분보유 기술 : Java, C, ReactBA, AA, DA, TADX 사업본부유통부분영업부 사업부영업 1팀영업 2팀마케팅팀영업/마케팅 사업본부영업 3팀', '2023블록체인 기반의 국가자격 · 훈련 디지털 배지시스템 구축 ‘과학기술정보통신부 장관상’MammothData 제42류 상표 등록 (제 40-1991414호)MammothData 제09류 상표 등록 (제 40-1991412호)보행자 돌발상황을 대처 할 수 있는 지능형 교통제어시스템- 특허 등록 (제 10-2555076호)국내∙ 외 투자 유치 (일본 하마지마, 유니온 커뮤니티)블록체인 기반의 디지털 이력서 증명서비스 구축 사업 수주 (한국인터넷진흥원)2022Software Processes 품질 인증 (SP2)INNOBIZ (기술혁신형 중소기업) 인정과학기술정보통신부 전문연구사업자 인정클라우드 기반 엔지니어링 빅데이터 및 엔지니어링 SW 활용 플랫폼 구축 (3개년)한국 축구 데이터 플랫폼 구축\n2021한국전자통신연구원 표창장 수상벤처기업(혁신성장유형) 인정MammothData 플랫폼 저작권 등록4세대 지능형 나이스 구축(응용SW 1,2 단계)_(3개년 사업)나이스 인사혁신처 간 시스템 연계를 통한 수당 이중지급 방지 체계 구축2020한

In [28]:
chat, index = set_chat()

while True:
    user_input = input("you: ")
    print(f"you: {user_input}")

    if user_input == "exit":
        break
    elif user_input.replace(" ", "") == "":
        print("chat: 질문이 없습니다. 정확한 질문을 해주시면 답변해 드리겠습니다.")
    else:
        chat_text = chat_input(
            chat=chat,
            index=index,
            user_input=user_input,
        )
        print(f"chat: {chat_text}")
        time.sleep(0.5)

타임소프트 회사소개서


ValidationError: 2 validation errors for EmbedRequest
input.str
  Input should be a valid string [type=string_type, input_value=[<langchain_community.doc...프트 회사소개서'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type
input.json-or-python[json=list[str],python=chain[is-instance[Sequence],function-wrap[sequence_validator()]]].0
  Input should be a valid string [type=string_type, input_value=<langchain_community.docu...t at 0x0000012058F3EF60>, input_type=PyPDFLoader]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type